In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
import time

In [3]:
driver = webdriver.Chrome('C://Users/LHG/chromedriver_win32/chromedriver.exe')


In [4]:
#itsub url
base_url="https://www.youtube.com/channel/UCdUcjkyZtf-1WJyPPiETF1g/videos"
video_list = []

In [5]:
#video list
driver.get(base_url)

body = driver.find_element_by_tag_name('body')

#page 넘기기
num_page_down = 30
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(5)
    num_page_down -= 1
    pass

html=driver.page_source
bsObject = BeautifulSoup(html,"html.parser")

#video가져오기
word="/watch?"
cnt=0
for link in bsObject.find_all('a'):
    try :
        if (link.get('href')[:len(word)] == word) & (cnt==0) :
            video_list.append(link.get('href'))
            cnt+=1
        else :
            cnt=0
    except TypeError :
        continue
    pass

In [44]:
#Data Frame
#video
video_columes =['Title','View','Likes']
video_data = pd.DataFrame(columns=video_columes)

#commnet
commnet_columes =['Title','Comment','Likes']
comment_data = pd.DataFrame(columns=commnet_columes)

In [45]:
#data crawling
youtube="https://www.youtube.com/"
for video in video_list :
    #video 탐색
    driver.get(youtube+video)
    body = driver.find_element_by_tag_name('body')
    num_page_down = 20#20
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_page_down -= 1
        pass
    
    video_html=driver.page_source
    video_bsObject = BeautifulSoup(video_html,"html.parser")
    #print(video_bsObject)
    
    #title
    title_find = video_bsObject.find_all(class_="ytp-title-link yt-uix-sessionlink ytp-title-fullerscreen-link")
    title=title_find[0].text.strip()#.encode('utf8')

  
    #hash tag
    hashtag_find = video_bsObject.find_all(class_="yt-simple-endpoint style-scope yt-formatted-string")
    cnt = 0
    tag_list=[]
    for word in hashtag_find :
        tag = word.text.strip()
        if (tag[0]=='#') :
            tag_list.append(tag)
            cnt+=1
        pass
    
    #view
    try :
        view_find = video_bsObject.find_all(class_="view-count style-scope yt-view-count-renderer")[0]
        temp_view = view_find.text.strip()
        view = temp_view[4:len(temp_view)-1].replace(",","").encode('utf8')
    except IndexError :
        view = 0
    
    #like
    try :
        like_find = video_bsObject.find_all(class_="hidden style-scope paper-tooltip")[1].text.strip()
        idx = like_find.find("/")-1
        video_like = int(like_find[:idx].replace(",",""))
    except IndexError:
        video_like=0
    if not cnt ==0 :
        tag_list = tag_list[:int(cnt/2)]
    
    video_data=video_data.append(pd.DataFrame(data=[[title,int(view),video_like]],columns=video_columes))
    print('=============================================================================================')
    print('영상제목',title,'조회수',int(view),'좋아요',video_like)
    comment_content=video_bsObject.find_all(class_="style-scope ytd-comment-renderer",id="content-text")
    comment_like=video_bsObject.find_all(id="vote-count-left")
        
    #모든 commnet 저장
    for i in range(len(comment_content)) :
        temp_content = comment_content[i].text.strip()
        try :
            temp_like = int(comment_like[i].text.strip())
        except ValueError :
            temp_txt = comment_like[i].text.strip()
            temp_like = float(temp_txt[:-1])
            unit=temp_txt[len(temp_txt)-1]
            multi=1000
            if unit == '만' :
                multi=10000
            temp_like=int(temp_like*multi)
        #commnet 저장
        print('댓글 내용',temp_content,'좋아요',temp_like)
        comment_data=comment_data.append(pd.DataFrame(data=[[title,temp_content,temp_like]],columns=commnet_columes))
        pass
    pass

영상제목 깊은 밤, 디스커버리 매장에서 생긴 일 "Night Travel" 조회수 264889 좋아요 5329
댓글 내용 요청이 많아서 미리해보는 나만의 맥북프로 만들기 링크입니다!
기본형 : https://apple.co/2Kq5wKx
고급형 : https://apple.co/2OkzGzI 좋아요 131
댓글 내용 아---- expensive 좋아요 280
댓글 내용 10:34 "---결제." 좋아요 56
댓글 내용 0:11
아닠ㅋㅋㅋㅋㅋ그 주스 흘리는 아저씨 따라한건갘ㅋㅋ 좋아요 51
댓글 내용 모토로라 폴더블 해주세여 ㅠㅠㅠ보자마자 구매욕구 넘치는데 ㅠㅠㅠㅠㅠㅠㅠ 좋아요 22
댓글 내용 0:50 사과 먹고있는거 커엽ㅋㅋㅋㅋㅋ 좋아요 23
댓글 내용 맥북 프로 존버 성공!!!!! 오예!!! ~잇섭님 말씀 처럼 16인치 고급형 가격은올라갔는데 15인치 고급형 대비 가성비가 좋아 진 것 같은 느낌이에요! 결론: 고민은 배송만 늦출뿐!! 좋아요 5
댓글 내용 들고다니면 운동 되것넹 ㅋㅋ 좋아요 96
댓글 내용 4:25 국밥한그릇~이 아니라 인텔이었네;; 좋아요 24
댓글 내용 역쉬 인트로장인ㅋㅋㅋㄱㅋ 좋아요 5
댓글 내용 오늘 기분 안좋으니까 잇섭 인트로 매드무비 보고잔다 좋아요 9
댓글 내용 아니 인트로 미틴 ㅋㅋㅋㅋㅋㅋ 너무 웃겨ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 좋아요 7
댓글 내용 8:09 근(?) 여기서도 좋아요 3
댓글 내용 잇섭형 사실 우리 헬창들은 그런거 신경안쓰는거 같아요.. 저는 무게 늘었다고 하니 가방에 넣고 중량 올리고 뛰기 더 좋아졌다 생각했어요... 허허허 좋아요 4
댓글 내용 유튭방송 안하고 영상편집 안해서 2014 얼리 버젼쓰는데 아직도 이상없이 한번도 고장없이 잘 돌아갑니다 
화질만 조금 떨어질뿐
이래서 맥북맥북하나봅니다 후훗
전 2030년에 새로운 맥북 구입을.. 큭 좋아요 6
댓글 내용 맥북 너무 비싸서 감당 불가.. 근데 진짜 디자인 너무 예쁨 좋아요 131
댓글 내용 진짜 기계를 전혀 모르는 사람도 이해하기 쉽게 잘 설명해주셔서 잘 보고

영상제목 구독자 100만 기념! 3세대 골드 버튼 언빡싱&감사한 마음으로 애플 에어팟 프로 10개 드립니다! 조회수 0 좋아요 0
영상제목 비싼게 좋은것 만은 아니다? 애플워치5 살 때 알루미늄vs스테인리스 선택법. 애플워치 5년차 사용자가 알려드립니다! (feat. 애플워치 시리즈5 언빡싱) 조회수 0 좋아요 0
영상제목 미개봉 태블릿이 12만원? 할인하면 꼭! 사세요. 최고의 갓성비 아마존 Fire HD 10 간단 사용기!(Amazon Fire HD 10 2017) 조회수 545011 좋아요 5904
댓글 내용 6:42 자막에 나온 'R'은 오타로 HDR을 지원하지 않습니다!
아마존 에코 버즈는 쪼금 더 써보고 올려볼게용 ㅎㅎ 좋아요 277
댓글 내용 머리젖고 옷젖는 아마~존입니다❤🐙 좋아요 1200
댓글 내용 잇섭 두뇌 : 소재 10% , 인트로 구성 80% , 애플10% 좋아요 886
댓글 내용 아니 ㅋㅋㅋ 아마존 인트로 실하냐곸ㅋ






솔직히 잇섭님은 인트로가 젤 꿀잼임 ㅋㅋ 좋아요 284
댓글 내용 워크맨 보고 인트로 만들었다는게 학계의 정설 좋아요 330
댓글 내용 포장이 무슨 다이소나 이런데서 팔거같음ㅋㅋ 막 태블릿 어딧어요 이러고 물어보면 걸려있을것 같고 ㅋㅋㅋ 좋아요 161
댓글 내용 케이스 2개면 본체 하나 라니 ㅋㅋㅋ 케이스가 비싼것인가 본체가 싼것인가 좋아요 292
댓글 내용 이거 동영상머신이랑 이북용으로는 최고임  ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 10만원초반대니까 개꿀이에요 좋아요 28
댓글 내용 화면비율 무엇? 꽉차서 좋다ㅋㅋㅋㅋㅋ 좋아요 183
댓글 내용 형 인트로 미쳤자나 ㅋㅋㅋㅋㅋ 좋아요 116
댓글 내용 진짜 포장지는 홈플러스 가전제품칸가면 걸려있을거같이 생김 좋아요 45
댓글 내용 잇섭님은 대형 유튜버인데도 하트랑 댓글 잘 달아줘서 좋아요👍😊 성실함이 돋보이십니다ㅋㅋ 좋아요 66
댓글 내용 1:37 순간 테블릿 던지는줄ㅋㅋ 좋아요 6
댓글 내용 카메라는 없는 걸로 할께요 ㅋㅋㅋㅋㅋㅋㄱㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㄱㅋㄱㅋㄱㄱㅋㄱㅋㄱㅋㄱㄱㅋ 좋아

영상제목 우리 럭시... 다시 켜질까요? 갈기갈기 분해된 갤럭시 폴드 재조립 도전! 조회수 288418 좋아요 5000
댓글 내용 M16 총기 분해결합 눈감고도 합니다. 좋아요 1300
댓글 내용 7:05 형 오른쪽 상단에 나사안꼈어 ㅠㅠ 좋아요 533
댓글 내용 누가보면 시간내에 조립 못하면 폭발하는 줄 알겠네ㅋㅋㅋ 좋아요 42
댓글 내용 8:54 상상체도 못한 정체ㅋㅋㅋㅋㅋㅋ 좋아요 105
댓글 내용 나사가 왜 남앜ㅋㅋ 좋아요 170
댓글 내용 중심에서 2시방향 나사 빠졌음 ㅋㅋㅋ
독일:나사가 모자란다 
일본:딱 맞다 
한국:남는다 
정상입니다 ㅋㅋㅋㅋ 좋아요 39
댓글 내용 이번 영상 요약 : 후욱후욱...쓰읍....후욱...후욱후욱... 좋아요 90
댓글 내용 나: 너무 많이 봤네... 이제 자야지
유튜브: 잇섭님이 갤포 재조립 다시 도전하신대 
나: ... 이것만 보고...ㅎㅎ 좋아요 134
댓글 내용 나사 1개 ㅋㅋㅋㅋㅋ 나중에 고장나면 레전드 ㅋㅋㅋㅋㅋ 좋아요 446
댓글 내용 아뉰ㄱㄱㄱㄱㄱㄱㄱ무슨 숨 소리가 ㄲ ㄱㅋㄱㄱㄱㄱㄱㄱㄱㄱ후욱후욱 좋아요 129
댓글 내용 숨소리가 무슨 야구동영상 배우보다 더 거칠어ㅋㅋㅋㅋㅋㅋㅋ
하지만 다 끝났다고 생각 한 순간...
8:48 진짜 당황의 시작 좋아요 21
댓글 내용 ㅋㅋㅋㅋㅋ나사 하나가 남았을때 느낌 OMR 밀려쓴 느낌이랑 비슷할듯해요 ㅋㅋㅋㅋㅋ 좋아요 23
댓글 내용 보는 내내 긴장했자낰ㅋㅋㅋㅋㅋ 잇섭님 숨소리 막 거칠어지고 ㅋㅋ 내가 조립하는줄!!
역시 대단합니다 최초일꺼임!! 좋아요 42
댓글 내용 폴드 조립 영상은 이분이 최초일듯ㅋㅌㅌㅋㅋㅋㅌㅋㅋㅋㅋ


+조립➡️➡️재조립!!💫 좋아요 346
댓글 내용 이형은 왜 이짓을 하고 있고 나는 왜 이걸 끝까지 ㅂ보고 있는가... 좋아요 16
댓글 내용 8:54 
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅌ소름돋는 순간이죠 난 완벽하게 재조립했다 생각했는데 남겨져 있는 나사를 본건ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 좋아요 33
댓글 내용 분해 할땐 -2398000원...
조립 할땐 +2398

댓글 내용 1 좋아요 3
댓글 내용 나사 1개 자리 알아냈어요.
7:10초에 보면 휴대폰 오른쪽 조금 상단에 나사 구멍 1개가 더 있어요. 나 좀 천재다. 좋아요 0
댓글 내용 ㅋㅋㅋㅋㅋㅋ 이 영상으로 폴드값 버셨을듯 좋아요 0
댓글 내용 거친 잇섭의 숨소리 ㅋㅋㅋ 좋아요 0
댓글 내용 와 마지막 너무 아찔해여 좋아요 0
댓글 내용 ㅋㅋㅋㅋㅋㅋㅋ 개꿀잼 ㅋㅋㅋ 폭탄 해체 하는것처럼 긴장 ㅋㅋ 나사1개는 무엇? ㅋㅋ 좋아요 0
댓글 내용 그의 거친 숨소리와 손놀림❤️❤️ 좋아요 1
댓글 내용 사장님이 아닌 형이 미쳤어요 ㅋㅋ 좋아요 1
댓글 내용 인트로 없어서 놀랐넼ㅋㅋㅋ 인트로 어디있엉ㅋㅋㅋ 좋아요 1
댓글 내용 보는 내내 긴장.. 좋아요 0
댓글 내용 실패각나오나 했는데 아쉽네요ㅎㅎ  그나저나 폴드 3주째쓰는 중인데 대박입니다 꼭사세요ㅎㅎ~ 좋아요 0
댓글 내용 노빠꾸 잇섭형님ㅋㅋㅋ 좋아요 0
댓글 내용 잇섭님 이제 삼성에서 스카우트 오시겠네요 부럽다 좋아요 1
댓글 내용 형 돈 남으면 나 기계식 청축 키보드 싼거 하나만ㅋㅋ(구걸충
혹시 진짜 주실수있으면 답장좀달아주세요
이메일로요)
frog3731@gmail.com 좋아요 0
댓글 내용 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ타임어택 컨텐츠는 처음보네 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 와우... 수고하셨습니다.... 좋아요 0
댓글 내용 7분20초  걸린 영상 ㅋ 좋아요 0
댓글 내용 와 켜질때 개놀람ㅋㅋ 좋아요 0
댓글 내용 이제 그냥 스마트폰 제조를 하셔도 될것 같아요 ㅋㅋ 좋아요 0
댓글 내용 마지막 ㅈㄴ 무섭네 ㅋㅋㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 잇섭님은 유튜브시작하기전에 반도체, 핸드폰 조립 IT 쪽 종사자이셨나요? 좋아요 1
댓글 내용 여윽시 잇섭님~~~~!!! 좋아요 1
댓글 내용 나사가 왜남...지..?ㅋㅋㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 분해 하실때 설마 재조립 영상도 올라올까 했는데.....역시.......항상 신제품 나오면 주변사람들한테 잇섭 유튜브 보라고 해요 짱짱👍🏻 좋아요 0
댓

댓글 내용 다음에 소니 WF-1000XM3도 언빡싱 해주세여ㅠㅠㅠㅠ 좋아요 0
댓글 내용 0:10
코카콜라에 펩시옷이라...
양쪽에서 광고받은건 아니겠죠? 좋아요 217
댓글 내용 단점:노캔했을때 뒤에서오는 차소리를 듣지못함 좋아요 13
댓글 내용 항상 멋진 영상 너무 잘보고 있어요 인스타도 팔로했어요 💜 좋아요 0
댓글 내용 오프닝미치겠넼ㅋㅋㅋㅋㅋㅋ 형 에어팟 프로 리뷰 기다렸어요!!! 좋아요 0
댓글 내용 저랑 평가가 같으시네요 ㅎㅎ 딱 그렇게 느꼈습니다. 음질을 원하는 사람은 소니로~ 좋아요 1
댓글 내용 와 포장에 오른쪽유닛 2갠지 몰랏네 형 눈썰미 조타 좋아요 220
댓글 내용 에어팟 프로를 샀는데 모다피가 왔어요ㅋㅋㅋ 좋아요 0
댓글 내용 2:05 이형 유니콘을 아네 즐감하세연~ 좋아요 0
댓글 내용 홍콩에서 직구했습니다
오는시간동안 폼팁 자체제작해서 써보려구요 좋아요 1
댓글 내용 인트로 시밬ㄲㅋㅋ 노콘 트롤ㅋㅋㅋㅋ 좋아요 0
댓글 내용 에어팟에 애플로고스티커 안들어 있는이유: 애초에 그 스티커는 폰이나 패드 등 필름 붙히기 전에 먼지 떼라고 나온 스티커라서 에어팟에는 없음 좋아요 167
댓글 내용 2:07 유니콘 https뭐야뭐야 좋아요 2
댓글 내용 영상 보시는 분들중 에어팟을 써보신 분, 혹은 유튜버님의 도움이 필요합니다..
오늘 받아서 써봤는데
노이즈 수음 모드 일 때, 몇초 간격으로 틱틱 거리는거 혹시 저만 그런가요???... 좋아요 0
댓글 내용 형 코카콜라 협찬받음?ㅋㅋㅋㅋ 좋아요 1
댓글 내용 여기가 그 유명한 seoul sexking 이 하는 유튜버 입니까? 좋아요 30
댓글 내용 형 딱원하던 부분이다 ㅋㅋㅋ
사랑해 ㅋㅋ 좋아요 124
댓글 내용 항상 이렇게 누구보다 빨리 올려주셔서 감사해요 ㅎㅎ 제가 살 수 없는 것들이지만 이렇게 영상으로 보면서 알아갈 수 있어서 항상 감사하게 느끼고있어요😊 좋아요 0
댓글 내용 형은 역시 재미도 재미지만
디테일이...👍 좋아요 1
댓글 내용 더 예뻐졌어요~ 귀여워 보여요~ 옛날 것보다 더 사고 

...

댓글 내용 와...영상이 재밌어서 10분이 그냥 지나가네..ㄷㄷ 👍 좋아요 0
댓글 내용 3:00 RTX2060 뒤에 등장 좋아요 3
댓글 내용 ㅌㅋㅌㅋㅋㅋㅋㅋ 촤하아아! 좋아요 0
댓글 내용 섭님 이 영상 너무 안 올라와서 잊고 계신 줄 알았어요 ㅋㅋㅋㅋㅋ 방금 구독 취소하려다 영상 보고 참았습니다 다음 영상 언제 올라오는 지 눈에 불을 켜고 기다리겠습니다 좋아요 0
댓글 내용 롤하고싶어요 카트랑..ㅜㅜ
(imseongmo7@naver.com) 좋아요 0
댓글 내용 인트로 뭐냐곸ㅋㅋㅋ 좋아요 0
댓글 내용 인트로 혼틈 스마트 키보드 폴리오 20만원 실화 ㅋㅋㅋㅋㅋㅋㅋㅋ




안녕하세요. 2060 나오는거 모르고 1070ti산 4K HDR 슈퍼 아몰레드 블랙말랑카우입니다
하아아아 좋아요 159
댓글 내용 오빠 나 컴 삼 301만원~~ 자랑하고 싶어서 왔엉^^ 좋아요 0
댓글 내용 2:08 잇섭님은 레드벨벳 중에 누가제일 좋아요? 좋아요 47
댓글 내용 저거 나눠준데!!! 나줘요!!!! ㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 첫 번째 영상 이후 엄청 기다렸는데 기다린 보람이 있네요😊 다시 구독하고 갑니다~! 좋아요 0
댓글 내용 형 오늘따라 잘생겨 보이네






농담이야. 좋아요 32
댓글 내용 ㅗㅜㅑ.. 저는 그냥 선정리 포기했죠 하ㅏ하ㅏㅏ.. 그래픽 카드 업그레이드 하고 하드 달면서 선이 하하ㅏ.. 좋아요 0
댓글 내용 잇섭님~~ 노트북용 1tb 용량인 ssd(맞나?) 추천해줘여ㅕ 좋아요 0
댓글 내용 만약에 라이젠쓸때 램4개로 풀뱅크돌리면 어떤가요 듀얼채널이 좋은가요?? 좋아요 2
댓글 내용 인트로 중독성 ㄷㄷ 5번째 돌려보는 중...

"겁나 싼거네!?" 좋아요 0
댓글 내용 와 알림에 역대 최저가 30만원대 게이...라고 떠서 어 스벌 머지? 하고 봤는데 잇섭님이네요 ㅋㅋㅋ 좋아요 27
댓글 내용 따라하면 엄크 ㅋㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 영상 하나보다가 어느새 3개째..ㅋㅋㅋㅋㅋㅋ
빠져든다아ㅏ
게이밍 컴퓨터 배그랑 롤! 돌아가는지 궁금해요! 좋아요

댓글 내용 오늘도 실망시키지 않는 인트로 ㅋㅋㅋ 좋아요 0
댓글 내용 형 udea모니터 리뷰좀 해줘~~♥
그게 가성비 갑이래 좋아요 1
댓글 내용 잇섭님 ! 진짜로 궁금했던건데 직업이 무엇인가요 ! 좋아요 1
댓글 내용 바로 구입하러갑니다!!
전기비때문에 엄청 고민했는데 겨울 도시가스 난로비 생각하면ㅠ왜고민했지ㅠ 좋아요 0
댓글 내용 형 오늘도 인트로재밌네요 역시 인트로형 유튜버 인트로만 보지 않고 풀영상 보는건 안비밀? 좋아요 0
댓글 내용 형님~ 콘센트 변경하는 방법 알려주세용~~ 좋아요 0
댓글 내용 플러그교체 어떻게하나용 ㅠㅠ 좋아요 0
댓글 내용 영상 잘 보고있습니다!!!! 
텐키리스 rgb 블루투스 기계식키보드 anne pro2 리뷰 부탁드려도될까요? 알리익스에서 75-85달러정도 인대 카일축 게이트론 축입니다!! 좋아요 0
댓글 내용 5:35 사용안할떄 18,000원
사용 28,000 좋아요 1
댓글 내용 그냥 이불덮고 전기장판에 지지는게...가장좋죸ㅋㅋㅋ 좋아요 0
댓글 내용 형 너무 재밌게 잘보고 있는데 우리집 컴이 상상을 못할 정도라 형이 까는 새제품을 볼때마다 한숨이 나와 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ 좋아요 1
댓글 내용 머리카락이 브로콜리 같아요!!!(진지) 좋아요 0
댓글 내용 인트로 유튜버 데스! 좋아요 0
댓글 내용 웃으면서 말씀하시는 게 매력적이네요 ㅎㅎㅎㅎ 좋아요 0
댓글 내용 좋은 리뷰 감사합니다~그런데 겨울이 끝물이에요....1월 말이에요....ㅠㅠ 좋아요 0
댓글 내용 지름 유발 요정님........ㅋㅋㅋ 좋아요 0
댓글 내용 인트로보고 순간 팔에 칼이.. 좋아요 0
댓글 내용 잇섭님! 영상잘봤습니다. 영상보면서 궁금한게있는데요! 스마트폰으로 이 제품을 조정할수있는 거리가 어느정도인가요?ㅎㅎㅎ 좋아요 0
댓글 내용 야생의 J210해머 풀아크릴을 발견했다! 좋아요 1
댓글 내용 형 나도 사랑해~ 좋아요 0
댓글 내용 5:18 에 뒤에 휴대폰 거치대 제품명이 뭔가요 깔끔해보이네요~ 좋아요 0
댓글 내용 전자제품

댓글 내용 블랙카우ㅋㄲㅋㅋㅋ 좋아요 0
댓글 내용 훗 난 산지 꾀 되어서 블랙 말랑카우는 아니라구.. ㅠㅠㅠㅠㅠ너무 싸잖아... 좋아요 1
댓글 내용 형 영상보고 아침 8시까지 나가서 오늘 사왔네요 ㅎ
좋은 정보덕분에 싸게 물건 잘샀어요 감사합니다!
(참고로 소니는 물량 통수를 쳐버렸다는....) 좋아요 0
댓글 내용 39주고 구매했던 흑우는 마음이 아픕니다... 2주됬는뎅 ㅠㅠ 좋아요 0
댓글 내용 오늘의 인트로: 지이이이인짜 블랙카우가(××찌 두번터치) 여기있다 좋아요 52
댓글 내용 저번주에 스위치 샀는데 
ㅠㅠㅠ 좋아요 0
댓글 내용 제스처너무이상해요ㅠㅠㅠㅠㅠㅠ 좋아요 0
댓글 내용 않희... 세뱃돈받아서 살려고 했는데 2월 3일까지 뭐냐고ㅠㅠ 좋아요 1
댓글 내용 좋은정보감사합니다!!! 항상 좋은정보 잘보고잇어요!! 구독 꾹!! 좋아요 0
댓글 내용 레알 그냥 사야하는 대박 핫딜 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 좋아요 5
댓글 내용 와 형님 감사합니다..ㅠㅠ 플스4프로 살려고 저번달부터 돈 뒤지게 모으는 중이였는데
돈이 부족해서 슬림을 사야하나 고민하는중이였는데 이렇게 할인 소식을 알려주셔서 정말 감사합니다😙 
(이영상 안봤으면 내일 슬림 지를뻔함!) 좋아요 0
댓글 내용 형 스팅어 언제 나와 ㅜㅜㅜㅜㅜㅜㅜㅜㅜ 좋아요 0
댓글 내용 중x나라 불구경 꿀잼엌ㅋㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 오늘 전화해보니까 하루만에 다 나갔다네ㅠㅠㅠㅠ 좋아요 0
댓글 내용 오늘도 인트로 보러 왔습니다ㅋㅋ 좋아요 6
댓글 내용 으..음메....음..메..ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ 좋아요 0
댓글 내용 와 소니가 통수를치네....아마이벤트기간동안 ps4를 구매못할꺼같아요. 이마트하고 하이마트 홈플러스 언제들어오냐고 전화돌려보니까 이제안들어온다...좀걸린다....등등 빼더라고요...와...이분 예언적중하셨네요..1분만에 품절 전부... 좋아요 16
댓글 내용 덕분에 주머니 괜찮아 졌습니다 ㅠㅠ 좋아요 0
댓글 내용 He is rich
흑우 좋아요 7
댓글 내용 I`m black cow. sib

댓글 내용 V40는 팀킬 안당해서 다행이다 ㅠㅠ 제발 플래그쉽만의 혜택을 주기를 바라는 1인입니다ㅠㅠ 좋아요 1
댓글 내용 스마트폰의전쟁?
ㅋㅋ제목이 웃기다 좋아요 0
댓글 내용 좀만있다가 LG 살껄ㅠㅠ 좋아요 0
댓글 내용 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
킹-치만 좋아요 0
댓글 내용 오늘도 인토로 보러 왔습니다!! 좋아요 0
댓글 내용 킹치만 ㅁㅊㅋㅋㅋㅋㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 차 비니루 포크나 칼로뜯는거 기대해볼게ㅋㅋㅋㅋ 좋아요 0
댓글 내용 크흠 ㅊㅂ님 언제 또 나오시나용..ㅎㅎ? 




부럽다 좋아요 0
댓글 내용 Q8산 사람은 웁니다 ㅜㅜㅜ 좋아요 0
댓글 내용 오~ 형 반속 좀 빠른가봐?? 휙~하고 던치면 바로 잡네...ㅂㄷㅂㄷ 좋아요 1
댓글 내용 잇섭의 클라쓰 빨리 들어왔는데 댓글 8개나있어 암튼 잇하!
궁금한게있는데 제폰이 v30인데 Q9보다 성능이 좋은가요? 좋아요 1
댓글 내용 msi 모니터 리뷰해주세요ㅠㅠ 좋아요 0
댓글 내용 하지만 이미 외면 받아버린 LG 스마트폰....
그리고 이미 스마트폰 시장을 독점해버린 
삼성과 애플... 좋아요 1
댓글 내용 근데 q9 배터리가 너무 빨리달아 ㅠㅜㅜ 좋아요 0
댓글 내용 Cpu i5 9400f 나왔는데 리뷰 부탁드려요☆☆ 좋아요 0
댓글 내용 그..그치만...이렇게라도 하지 않으면 잇섭짱은 나한테 관심도 없는걸..... 좋아요 2
댓글 내용 A7 2018 쓰고있는데 Q9 쓰고싶다... 좋아요 1
댓글 내용 잇섭님 애플 맥북 에어는 리뷰 안하시나영?? 좋아요 1
댓글 내용 지금 821 쓰고 있어서 아는데, 안쓰러울 정도로 쓰로틀링 걸어버리면 안투투 점수 5자리도 나옵니다. 반대로 쿨링을 해주면 날라다니고요 좋아요 0
댓글 내용 ㅋㅋㅋㅋㅋㅋㅋㅋ 좋아요 0
댓글 내용 오늘도 인트로 보는맛에 옵니다 좋아요 4
댓글 내용 잇섭님! 저희도 구독자 애칭 만들어요~! 좋아요 0
댓글 내용 언박싱 감사합니다! 좋아요 0
댓글 내용 이렇게 사람들 속이고나면 또 헿칭챤머그따헿헿 거리면서 똥폰만들겠지? 좋아요 2
댓

ValueError: invalid literal for int() with base 10: ''

In [46]:
comment_data.to_csv('../comment.csv')
video_data.to_csv('../video.csv')